In [1]:
'''

'''
import os
import sys
import re
import glob
import gc
import json
import copy
import pickle
import warnings

import datetime as dt

import pyodbc
import pandas as pd
import numpy as np
import geocoder

import lightgbm as lgb
import xgboost as xgb
import catboost as cat


from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import RandomOverSampler


from folium import Map, Marker
from folium.plugins import MarkerCluster

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

from shapely.geometry import shape
from geopy.distance import geodesic

from IPython.display import display as disp
from IPython.display import display_markdown


warnings.filterwarnings('ignore')
pd.options.display.max_rows = 300
pd.options.display.max_columns = None

dispmd = lambda txt: display_markdown(txt, raw=True)

In [2]:
FOLD_SIZE  = 5

TARGET_COL = 'money_room'

MODEL_PATH = 'models'

CONN_DICT = {
    'DRIVER': '{ODBC Driver 18 for SQL Server}',
    'SERVER': '[server_name}',
    'DATABASE': '[dbname]',
    'UID': '[username]',
    'PWD': '[PASSWORD]',
    'trusted_connection': 'no',
    'TrustServerCertificate': 'no',
    'encrypt': 'no'
}


In [3]:
description_path = 'data_definition_manu.xlsx'
df_data_desc = pd.read_excel(description_path, sheet_name='①データ定義書')
disp(df_data_desc.head(10))

df_data_desc = df_data_desc[~df_data_desc['DataType'].isna()]
df_data_desc = df_data_desc[['本番データ特徴量名', 'DataType']].set_index('本番データ特徴量名')

col_map = df_data_desc.to_dict()['DataType']
print(col_map)

,No.,本番データ特徴量名,DataType,Drop,特徴量の意味,補足
0,1,target_ym,int32,NaN,対象年月,各レコードの抽出対象年月日 yyyymm
1,2,money_room,int64,NaN,賃料(代表),対応する物件の賃料。目的変数となる。
2,3,building_id,string,NaN,棟ID,AUTO_INCREMENT、UNSIGNED 確認事項参照
3,4,building_status,category,NaN,状態,1: 棟が存在する、9: 棟が存在しない
4,5,building_create_date,datetime,NaN,作成日時,データ作成日時
5,6,building_modify_date,datetime,NaN,修正日時,データ修正日時
6,7,building_type,category,NaN,建物種別,"1: マンション, 3: アパート, その他: 欠損"
7,8,building_name,string,NaN,建物名,NaN
8,9,building_name_ruby,string,NaN,建物名フリガナ,NaN
9,10,homes_building_name,string,NaN,HOME'S 建物名,掲載時の物件名


{'target_ym': 'int32', 'money_room': 'int64', 'building_id': 'string', 'building_status': 'category', 'building_create_date': 'datetime', 'building_modify_date': 'datetime', 'building_type': 'category', 'building_name': 'string', 'building_name_ruby': 'string', 'homes_building_name': 'string', 'homes_building_name_ruby': 'string', 'unit_count': 'int16', 'full_address': 'string', 'lon': 'float', 'lat': 'float', 'building_structure': 'category', 'total_floor_area': 'float', 'building_area': 'float', 'floor_count': 'int8', 'basement_floor_count': 'int8', 'year_built': 'int32', 'building_land_area': 'float', 'land_area_all': 'float', 'unit_area_min': 'float', 'unit_area_max': 'float', 'building_land_chimoku': 'category', 'land_youto': 'category', 'land_toshi': 'category', 'land_chisei': 'category', 'land_area_kind': 'category', 'land_setback_flg': 'category', 'land_setback': 'float', 'land_kenpei': 'int16', 'land_youseki': 'int16', 'land_road_cond': 'category', 'land_seigen': 'string', 'bu

In [4]:
def ret_dtype(val):
    if val == 'string':
        return str
    elif val == 'float':
        return np.float32
    elif val[:3] == 'int':
        if int(val[3:]) == 8:
            return pd.Int8Dtype()
        elif int(val[3:]) == 16:
            return pd.Int16Dtype()
        elif int(val[3:]) == 32:
            return pd.Int32Dtype()
        else:
            return pd.Int64Dtype()
    else:
        return val
        
feat_cols_dict = {k: ret_dtype(v) for k, v in col_map.items() if v not in ['datetime', 'date']}

## 外れ値(価格の誤入力)と思われるものの記録
* 440988
* 397389
* 203618
* 416163
* 394463
* 127129
* 127125
* 558572
* 300281
* 300280
* 358515: 62,000の誤りと思う
* 78395
* 481129
* 384632

## シェアハウス等で特殊なもの
* 54552
* 570632

## house_areaの値がおかしいもの

* 285800: 1/10となっている？
* 107987: 1/10となっている？
* 125096: 1/3となっている?
* 410743: 1m^2となっている。
* 446734: 28.98000m^2と思われる
* 43185: 29.70m^2
* 165655: 50.000000m^2と思われる
* 405859: 25.00m^2と思われる
* 146486: 44.00m^2と思われる
* 139573: 85.870003m^2と思われる

In [5]:
path = './train.csv.gz'

read_opts = {
    'encoding': 'utf8', 
    'dtype': feat_cols_dict,
    'parse_dates': [
        'building_create_date',
        'building_modify_date',
        'reform_date',
        'renovation_date',
        'snapshot_create_date',
        'new_date',
        'snapshot_modify_date'
    ]
}

df_train = pd.read_csv(path, **read_opts)
dispmd(f'### DataFrame Shape = ROWS: {df_train.shape[0]:,}, COLUMNS: {df_train.shape[1]}')
disp(df_train.head())

### DataFrame Shape = ROWS: 584,507, COLUMNS: 152

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,202007,38000,a000002,1,2014-06-27 21:18:41,2024-05-25 00:45:07,3,Plaisirりんくう,NaN,プレジール りんくう,プレジールりんくう,22,大阪府泉佐野市笠松1丁目6-51,135.309296,34.411514,1,NaN,NaN,2,<NA>,200210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,321101/310201/320901/321001/210202/210301/210101,23810544,202,NaN,2,2.0,5,1,23.180000,120,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaT,NaN,230801/310501/220301/290901/230101/240104/2205...,1344300002216,2020-07-30,2020-07-30,2020-07-30 16:21:04,2020-08-06 00:00:00,1,1,3102,0,2,202,598,44,27,213,笠松,NaN,1-6-51,123869683.0,487123547.0,南海線,泉佐野,NaN,0,1040,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.180000,0,NaN,2,5,1,20,3000,3,NaN,<NA>,<NA>,木下の賃貸 友の会費,1760,カギ交換代,16500,木下の消毒サービス,26400,8800,2,1,0,1,NaN,3,3,202009,NaN,<NA>,NaN,NaN,<NA>,NaN,239,1280,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,110702/210202/220101/220201/220301/230101/2308...,NaN,NaN,<NA>,NaN
1,202107,103000,a000004,1,2017-02-05 20:43:26,2024-08-17 12:12:51,1,向原パークハイツ,NaN,向原パークハイツ,ムカイハラパークハイツ,<NA>,東京都板橋区向原1丁目11-1,139.684555,35.741833,4,NaN,NaN,3,<NA>,197612,NaN,NaN,44.099998,44.099998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,210101/330501/320901/321001/210301/210201,33983775,110,NaN,1,4.0,5,2,47.700001,230,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaT,NaN,340101/253401/220401/290901/220201/230202/2943...,1279070015508,2021-07-09,2021-07-09,2021-07-30 23:36:05,2021-08-06 00:00:00,1,0,3101,0,<NA>,110,173,36,13,119,向原１丁目,11-1,NaN,128659541.0,502875929.0,東京メトロ有楽町線,小竹向原,NaN,<NA>,320,東京メトロ有楽町線,千川,NaN,NaN,400,東京地下鉄副都心線 小竹向原 徒歩4分,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.700001,0,NaN,1,5,2,30,6000,3,NaN,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,22000,2,3,300,<NA>,NaN,2,1,<NA>,板橋区立向原小学校,291,NaN

## SQL Serverに格納した国土数値情報の利用

0. [地価公示][地価公示](L01-24): 特徴量として利用する際には、次のものを利用しました。
    * 最近傍の公示価格をそのまま利用
    * 最近傍の建築可能面積(`建ぺい率 * 容積率 * 公示価格`)
    * 最も距離が近いもの上位3地点の公示価格を逆距離加重して算出
1. [1kmメッシュ別将来推計人口（H30国政局推計）][1kmメッシュ別将来推計人口（H30国政局推計）](1km_mesh_2018): 2020年度の人口推計値のうち
2. [医療機関][医療機関](P04_2020): 半径2km以内の診療所、病院、歯科医院の件数をカウントしました。 
3. [小学校区][小学校区](A27_23): 特徴量としては、小学校の学校コードと距離を使用しました。
    * 全ての地方公共団体がデータを提供しているわけではないため、学区情報がない自治体が存在する(e.g. 品川区)
    * 地点によっては、学区が選択制となっているため、複数の小学校に通うことができる。この場合は、仕方がないので、(直線距離で)距離が近いものとしました。
4. [学校][学校](P29_23): 教育施設の充実度として、半径2km以内の小学校、中学校、中等教育学校、高等学校、大学、義務教育学校、幼稚園・こども園の数をそれぞれカウントしました。
5. [駅別乗降客数][駅別乗降客数](S12-23): このデータを使って棟情報と駅との距離を3km以内とし、最も近い駅の`S12_001g`を特徴量としました。

[駅別乗降客数]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-S12-2022.html
[学校]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P29-2023.html
[1kmメッシュ別将来推計人口（H30国政局推計）]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-mesh1000h30.html
[小学校区]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-A27-2023.html
[地価公示]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-L01-2024.html
[医療機関]: https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P04-2020.html

In [6]:
conn = pyodbc.connect(';'.join([f'{k}={v}' for k, v in CONN_DICT.items()]))
cursor = conn.cursor()

query = '''
DECLARE @P FLOAT = 8; -- 距離減衰パラメータ    

WITH TEMP1 AS (
    SELECT DISTINCT
        公示地価との距離.*,
        公示地価.地価2024,
        CASE 
            WHEN 公示地価との距離.distance = 0 THEN 1.0
            ELSE POWER(公示地価との距離.distance, @P)
        END AS Weight,
        公示地価.建蔽率,
        公示地価.容積率,
        CASE
            WHEN 公示地価.建蔽率 = 0 OR 公示地価.容積率 = 0 THEN NULL
            ELSE
            公示地価.地価2024 / (公示地価.建蔽率 * 公示地価.容積率 / 10000.00)
        END AS 地価,
        最寄り駅名,
        大分類,
        最寄り駅迄の道路距離,
        詳細
    FROM
        dbo.公示地価との距離
    LEFT JOIN dev.dbo.公示地価 ON
        公示地価.行政区域コード + 公示地価.用途区分 + 公示地価.連番 = 公示地価との距離.行政区域コード + 公示地価との距離.用途区分 + 公示地価との距離.連番
    WHERE
        公示地価.地価2024 > 0
), 逆距離加重 AS (
    SELECT
        building_id,
        CASE 
            WHEN SUM(Weight) = 0 THEN NULL -- 距離がすべて無限大の場合（安全策）
            ELSE SUM(Weight * 地価2024) / SUM(Weight) 
        END AS 逆距離加重地価2024,
        CASE 
            WHEN SUM(Weight) = 0 THEN NULL -- 距離がすべて無限大の場合（安全策）
            ELSE SUM(Weight * 地価) / SUM(Weight) 
        END AS 逆距離加重地価
    FROM 
        TEMP1
    GROUP BY
        building_id
), 最近傍 AS (
    SELECT
        building_id,
        地価2024,
        地価,
        distance,
        建蔽率,
        容積率,
        最寄り駅名,
        大分類,
        最寄り駅迄の道路距離
    FROM
        TEMP1
    WHERE
        順序 = 1
)
SELECT DISTINCT
    最近傍.*,
    逆距離加重.逆距離加重地価2024,
    逆距離加重.逆距離加重地価
FROM
    最近傍
INNER JOIN 逆距離加重 ON
    最近傍.building_id = 逆距離加重.building_id
ORDER BY
    building_id ASC
;
'''

df_geo = pd.read_sql_query(query, conn)

df_geo = df_geo[['building_id', 'distance', '地価2024', '建蔽率', '容積率', '地価', '最寄り駅名',
                #  '逆距離加重地価2024', '逆距離加重地価'
                 ]]
df_geo['最寄り駅名'] = df_geo['最寄り駅名'].astype('category')

disp(df_geo.head())

query = '''
SELECT 
    棟情報_人口.building_id,
    棟情報_人口.MESH_ID,
    km_mesh_pop.PTN_2020,
    km_mesh_pop.PTA_2020,
    km_mesh_pop.PTB_2020,
    km_mesh_pop.PTC_2020,
    km_mesh_pop.PTD_2020,
    km_mesh_pop.PTE_2020,
    km_mesh_pop.PTA_2020 / km_mesh_pop.PTN_2020 AS PTA_2020_perN,
    km_mesh_pop.PTB_2020 / km_mesh_pop.PTN_2020 AS PTB_2020_perN,
    km_mesh_pop.PTC_2020 / km_mesh_pop.PTN_2020 AS PTC_2020_perN,
    km_mesh_pop.PTD_2020 / km_mesh_pop.PTN_2020 AS PTD_2020_perN,
    km_mesh_pop.PTE_2020 / km_mesh_pop.PTN_2020 AS PTE_2020_perN    
FROM
    dev.gis.棟情報_人口
LEFT JOIN dev.dbo.km_mesh_pop ON 
    棟情報_人口.MESH_ID = km_mesh_pop.MESH_ID
LEFT JOIN dev.gis.HF_MESH_COUNT ON 
    HF_MESH_COUNT.MESH_ID = km_mesh_pop.MESH_ID 
;
'''

df_pop = pd.read_sql_query(query, conn)
disp(df_pop.head())


query = '''
-- 半径2km以内の医療機関
SELECT 
    *    
FROM
    dev.dbo.近隣医療機関
;
'''

df_hf = pd.read_sql_query(query, conn)
disp(df_hf.head())


query = '''
-- 半径1km以内の教育施設

WITH TEMP1 AS (
    SELECT
        building_id,
        COUNT(*) AS 件数
    FROM
        dev.dbo.棟情報
    GROUP BY
        building_id
), 近隣教育機関 AS (
    SELECT
        棟情報.building_id,
        SUM(IIF(学校.P29_003 = '16001', 1, 0)) AS 小学校,
        SUM(IIF(学校.P29_003 = '16002', 1, 0)) AS 中学校,
        SUM(IIF(学校.P29_003 = '16003', 1, 0)) AS 中等教育学校,
        SUM(IIF(学校.P29_003 = '16004', 1, 0)) AS 高等学校,
        -- SUM(IIF(学校.P29_003 = '16005', 1, 0)) AS 高等専門学校,
        -- SUM(IIF(学校.P29_003 = '16006', 1, 0)) AS 短期大学,
        SUM(IIF(学校.P29_003 = '16007', 1, 0)) AS 大学,
        -- SUM(IIF(学校.P29_003 = '16011', 1, 0)) AS 幼稚園,
        -- SUM(IIF(学校.P29_003 = '16012', 1, 0)) AS 特別支援学校,
        -- SUM(IIF(学校.P29_003 = '16013', 1, 0)) AS 幼保連携型認定こども園,
        SUM(IIF(学校.P29_003 = '16014', 1, 0)) AS 義務教育学校,
        SUM(IIF(学校.P29_003 IN ('16011', '16013'), 1, 0)) AS [幼稚園・こども園]
    -- ,
    -- SUM(IIF(学校.P29_003 = '16015', 1, 0)) AS 各種学校,
    -- SUM(IIF(学校.P29_003 = '16016', 1, 0)) AS 専修学校
    FROM
        dev.dbo.棟情報
    INNER JOIN dev.gis.学校 ON 
        棟情報.[geopoint].STDistance(学校.geometry) <= 2000
    WHERE
        学校.P29_007 = '1' -- 休校を除く
        AND 学校.P29_003 NOT IN ('16015', '16016') -- 各種学校・専修学校
    GROUP BY
        棟情報.building_id
)
SELECT
    近隣教育機関.building_id,
--    近隣教育機関.小学校 / TEMP1.件数 AS 小学校,
--    近隣教育機関.中学校 / TEMP1.件数 AS 中学校,
--    近隣教育機関.中等教育学校 / TEMP1.件数 AS 中等教育学校,
    近隣教育機関.高等学校 / TEMP1.件数  AS 高等学校,
--    近隣教育機関.大学 / TEMP1.件数 AS 大学,
    近隣教育機関.義務教育学校 / TEMP1.件数 AS 義務教育学校,
    近隣教育機関.[幼稚園・こども園] / TEMP1.件数 AS [幼稚園・こども園]
FROM
    近隣教育機関
LEFT JOIN TEMP1 ON 
    TEMP1.building_id = 近隣教育機関.building_id
ORDER BY
    近隣教育機関.building_id
;
'''

df_ed = pd.read_sql_query(query, conn)
disp(df_ed.head())



query = '''
SELECT DISTINCT
    building_id,
    S12_001,
    S12_001g,
    駅までの距離
FROM
    dev.dbo.駅までの距離
WHERE
    順位 = 1
;
'''

df_station = pd.read_sql_query(query, conn)
disp(df_station.head())


,building_id,distance,地価2024,建蔽率,容積率,地価,最寄り駅名
0,a000000,0.059561,63600.0,60.0,200.0,53000.000000,近鉄弥富
1,a000001,0.464198,57600.0,60.0,200.0,48000.000000,柳原
2,a000002,0.284153,73900.0,60.0,200.0,61583.333333,泉佐野
3,a000003,0.398928,94200.0,60.0,200.0,78500.000000,名鉄味美
4,a000004,0.457499,513000.0,60.0,200.0,427500.000000,千川


,building_id,MESH_ID,PTN_2020,PTA_2020,PTB_2020,PTC_2020,PTD_2020,PTE_2020,PTA_2020_perN,PTB_2020_perN,PTC_2020_perN,PTD_2020_perN,PTE_2020_perN
0,a381283,53393448.0,9243.6875,1117.5916,6147.2484,1978.8475,1106.057,705.1967,0.120903,0.665021,0.214076,0.119655,0.07629
1,a315852,53393448.0,9243.6875,1117.5916,6147.2484,1978.8475,1106.057,705.1967,0.120903,0.665021,0.214076,0.119655,0.07629
2,a527461,53393448.0,9243.6875,1117.5916,6147.2484,1978.8475,1106.057,705.1967,0.120903,0.665021,0.214076,0.119655,0.07629
3,a333429,53393448.0,9243.6875,1117.5916,6147.2484,1978.8475,1106.057,705.1967,0.120903,0.665021,0.214076,0.119655,0.07629
4,a525052,53393448.0,9243.6875,1117.5916,6147.2484,1978.8475,1106.057,705.1967,0.120903,0.665021,0.214076,0.119655,0.07629


,building_id,病院,診療所,歯科医院
0,a298383,4,37,28
1,a379882,9,162,116
2,a143529,15,178,141
3,a365872,12,257,219
4,a543722,12,321,277


,building_id,高等学校,義務教育学校,幼稚園・こども園
0,a000000,0,0,0
1,a000001,0,0,0
2,a000002,0,0,0
3,a000003,0,0,0
4,a000004,0,0,0


,building_id,S12_001,S12_001g,駅までの距離
0,a419017,六甲道,007004,474.501910
1,a406160,香里園,006699,863.367403
2,a178358,水戸,002319,1300.009010
3,a203250,真岡,002224,1658.245239
4,a494725,西葛西,003890,595.537597


In [7]:
query = '''
WITH TEMP1 AS (
    SELECT 
        小学校区情報.building_id,
        小学校区情報.A27_003 AS 学校コード,
        小学校区情報.小学校名,
        V_学校.GEOPOINT.STDistance(小学校区情報.geopoint) AS 距離
    FROM
        dev.dbo.小学校区情報
    LEFT JOIN dev.gis.V_学校 ON
        V_学校.学校コード = 小学校区情報.A27_003
), TEMP2 AS (
    SELECT
        building_id,
        学校コード,
        小学校名,
        距離,
        RANK() OVER(PARTITION BY building_id ORDER BY 距離 ASC) AS 順位
    FROM
        TEMP1
)
SELECT
    building_id,
    学校コード,
    小学校名,
    距離 AS 小学校_距離
FROM
    TEMP2
WHERE
    順位 = 1
'''

df_elems = pd.read_sql_query(query, conn)
disp(df_elems.head())

cursor.close()
conn.close()

,building_id,学校コード,小学校名,小学校_距離
0,a002608,B111220800017,所沢市立所沢小学校,643.353794
1,a002609,B127210008387,摂津市立鳥飼北小学校,309.660365
2,a002610,B117220100161,金沢市立鞍月小学校,233.006025
3,a002612,B113212000207,練馬区立練馬第二小学校,191.337153
4,a002615,B104221010074,仙台市立通町小学校,469.149455


In [9]:
df_tag_master = pd.read_excel(description_path, sheet_name='⑤設備情報シート', dtype={
    'タグID': str,
    'タグ内容': str,
})

disp(df_tag_master.head(10))


,No.,タグID,タグ内容
0,1,110101,楽器相談
1,2,110102,楽器不可
2,3,110201,事務所可
3,4,110202,事務所不可
4,5,110301,二人入居可
5,6,110302,二人入居不可
6,7,110401,男性限定
7,8,110402,女性限定
8,9,110501,単身者限定
9,10,110502,単身者希望


## 相当怪しそうなもの
* 360787	202307	a106852	MGAハウス	MGAハウス	269485	東京都世田谷区太子堂2丁目2-23	74.000000	31.469999
* 005721	202307	a082528	PALAZZO	パラッツオ南太田	596996	神奈川県横浜市南区南太田1丁目19-7	246.0	246.440002	=> room_floorが5階でroom_kaisuが6階なのでおそらくメゾネットタイプである。ただ3LDKなのは広すぎるようにも思う

## 修正対象
* 200829	202307	a263335	ランドマーク	ランドマーク	324481	静岡県三島市梅名101-5	312.0	78.199997	
* 232292	202301	a279060	メゾン・ラフィネ1	メゾンラフィネI	223733	埼玉県上尾市大字領家105-1	258.0	58.860001	
* 237700	202307	a454636	Coco覚王山	Coco覚王山2	191157	愛知県名古屋市千種区観月町2丁目70	101.0	25.680000
* 170551	202301	a419953	フラワービレッジ	フラワービレッジ	196742	愛知県名古屋市南区三吉町4丁目43-3	187.000000	87.419998	1
* 326377	202201	a326469	セレブYAGA	セレブ・YAGA	193676	東京都目黒区目黒本町3丁目13-22	64.580002	24.360001		

## 再度確認するもの
* 226145	202307	a216813	カサ・サンブラス	カサ・サンブラス	142023	東京都杉並区松庵3丁目33-2	45.000000	16.430000
* 318965	202207	a468503	レジデンス上前津	S-RESIDENCE上前津	134245	愛知県名古屋市中区上前津2丁目5-16	65.000000	22.500000	
* 186698	202207	a011026	ステイブルアイ	ステイブル アイ	111658	東京都品川区西品川3丁目9-10	46.000000	17.730000

### 問題なそう
* 363186	202301	a535146	LAMIACASA	LA MIA CASA	103731	岡山県岡山市南区東畦155-18	84.000000	34.980000 => unit_areaが誤りか？
* 123577	202301	a193924	KDXレジデンス両国	KDXレジデンス両国	194000	東京都墨田区亀沢2丁目19-13	53.000000	25.480000 => unit_areaが誤り？・前回値を使用する。

In [10]:
def preprocessing(df: pd.DataFrame, feat_cols_dict: dict, test=False) -> tuple[pd.DataFrame, dict]:
    '''
    '''
    out_cols_dict = copy.copy(feat_cols_dict)
    
    ## 0. 設備情報
    df_statuses = pd.DataFrame(df['statuses'].str.split('/', expand=True).stack(), columns=['unit_tag'])
    df_statuses = df_statuses.droplevel(level=1)
    df_statuses = df_statuses.reset_index()

    df_statuses = df_statuses.merge(df_tag_master[['タグID', 'タグ内容']], 
                                    left_on='unit_tag', right_on='タグID',
                                    how='inner')
    df_statuses['statuses'] = True
    df_statuses = df_statuses.set_index(['index', 'タグ内容'])['statuses'].unstack(level=1)

    for col in df_statuses.columns:
        df_statuses[col] = df_statuses[col].astype(pd.Int8Dtype())

    if '建築条件付き' in df_statuses.columns:
        df_statuses.drop(columns='建築条件付き', inplace=True)


    disp(df_statuses.head(10))
    
    # 1. floor_plan_codeは最初の数字が部屋数の1桁目となっているので、
    # 　　2から3文字目を取り出す
    col = 'floor_plan_code'
    df[col] = df[col].str.slice(1, 3)
    df[col] = df[col].map({
        '10': 'R', 
        '20': 'K',
        '30': 'DK',
        '40': 'LK',
        '50': 'LDK'
    })
    # df[col] = df['room_count'].astype(str) + df[col] 
    df[col] = df[col].astype('category')
    out_cols_dict.update({col: 'category'})

    col = 'madori_kind_all'
    df[col] = df[col].map({
        '10': 'R',
        '20': 'K',
        '25': 'SK',
        '30': 'DK',
        '35': 'SDK',
        '40': 'LK',
        '45': 'SLK',
        '50': 'LDK',
        '55': 'SLDK',
    })
    
    # df[col] = df['madori_number_all'].astype(str) + df[col] 
    df[col] = df[col].astype('category')
    out_cols_dict.update({col: 'category'})

        
    # 2. target_ymからyearとmonthを取り出し、
    col = 'target_ym'
    df[f'{col}_year'] = df[col] // 100    
    df[f'{col}_month'] = df[col] % 100
    
    out_cols_dict.update({f'{col}_year': pd.Int16Dtype()})
    out_cols_dict.update({f'{col}_month': pd.Int8Dtype()})    
    del out_cols_dict[col]

    # 3. post1とpost2を結合して、カテゴリー変数として使用する。
    # また、post1単独でカテゴリー変数として使用する
    df['post1'] = pd.to_numeric(df['post1']).astype(pd.Int16Dtype()).fillna(0)
    df['post2'] = pd.to_numeric(df['post2']).astype(pd.Int16Dtype()).fillna(0)
    
    df['post1'] = df['post1'].astype(str).str.pad(3, fillchar='0')
    df['post2'] = df['post2'].astype(str).str.pad(4, fillchar='0')

    df['post'] = df['post1'] + df['post2']
    df['post'] = df['post'].astype('category')
    df['post1'] = df['post1'].astype('category')

    out_cols_dict.update({'post': 'category'})
    out_cols_dict.update({'post1': 'category'})
    del out_cols_dict['post2']

    # 4. addr1とaddr22を結合して、カテゴリー変数として使用する。
    # また、post1単独でカテゴリー変数として使用する
    df['addr1_1'] = df['addr1_1'].astype(str).str.pad(2, fillchar='0')
    df['addr1_2'] = df['addr1_2'].astype(str).str.pad(3, fillchar='0')

    df['addr1'] = df['addr1_1'] + df['addr1_2']
    df['addr1'] = df['addr1'].astype('category')
    out_cols_dict.update({'addr1': 'category'})


    df['addr1_1'] = df['addr1_1'].astype('category')
    out_cols_dict.update({'addr1_1': 'category'})

    del out_cols_dict['addr1_2']

    # 4. yyyymm形式で表されているものをtarget_ymからの経過月数とする。
    target_cols = [
        'year_built',
        'reform_exterior_date',
        'reform_common_area_date',
        'reform_wet_area_date',
        'reform_interior_date',
    ]

    for col in target_cols:
        df[col] = (df['target_ym_year'] - df[col] // 100) * 12 + (df['target_ym_month'] - df[col] % 100)
        df[col] = df[col].astype(pd.Int16Dtype())

    # 6. 駐車場の内税・外税処理 (2024-10-24)
    target_cols = [
        'money_kyoueki',
        'parking_money'
    ]

    for col in target_cols:
        df.loc[df[f'{col}_tax'] == '1', col] = np.ceil(df[col] * 1.10)
        del out_cols_dict[f'{col}_tax']

    # 7. 空き率  (2024-10-24)
    df['empty_number'] = df['empty_number'] / df['unit_count']

    # house_areaが極端に大きくおかしい物件を修正
    df.loc[(df['house_area'] >= 10 * df['unit_area']) & (df['house_area'] > 100), 
    'house_area'] = df['unit_area']

    # house_areaが極端に小さくおかしい物件を修正
    df.loc[(10 * df['house_area'] <= df['unit_area']) & (df['house_area'] < 20), 
    'house_area'] = df['unit_area']
    ## 8707から追加 ここまで

    # f. 使用しないstringやdatetime columns
    drop_cols = [
        # string
        'building_id',
        'building_name',
        'building_name_ruby',
        'homes_building_name',
        'homes_building_name_ruby',
        'full_address',
        'land_seigen',

        'money_rimawari_now',
        'unit_area',
        
        'eki_name1'
    ]

    for col in drop_cols:
        del out_cols_dict[col]

    # # 公示地価の紐付け
    df = df.merge(df_geo, on=['building_id'], how='left')
    out_cols_dict.update({'distance': np.float32, 
                          '地価2024': pd.Int32Dtype(),
                          '建蔽率': pd.Int8Dtype(),
                          '容積率': pd.Int16Dtype(),
                          '地価': np.float32,
                          
                        #   '逆距離加重地価2024': np.float32,
                        #   '逆距離加重地価': np.float32,
                        #   '最寄り駅名': 'category'
                          })
    
    df = df.merge(df_pop, on=['building_id'], how='left')
    out_cols_dict.update({'PTN_2020': np.float32, 
                          'PTA_2020': np.float32,
                          'PTB_2020': np.float32,
                          'PTC_2020': np.float32,
                          'PTD_2020': np.float32,
                          'PTE_2020': np.float32,
                          'PTA_2020_perN': np.float32,
                          'PTB_2020_perN': np.float32,
                          'PTC_2020_perN': np.float32,
                          'PTD_2020_perN': np.float32,
                          'PTE_2020_perN': np.float32,
                          # Baselineから追加 
                          })
    
    df = df.merge(df_hf, on='building_id', how='left')
    out_cols_dict.update({
                          '病院': pd.Int16Dtype(),
                          '診療所':  pd.Int16Dtype(),
                          '歯科医院': pd.Int16Dtype(),
    })
    
    df = df.merge(df_ed, on='building_id', how='left')
    out_cols_dict.update({col: pd.Int16Dtype() for col in df_ed.columns[1:]})

    df = df.merge(df_station, on='building_id', how='left')
    df['S12_001g'] = df['S12_001g'].astype('category')
    out_cols_dict.update({
        'S12_001g': 'category',
        '駅までの距離': np.float32
    })

    # 2024-11-06
    df['floor_ratio'] = df['room_kaisuu'] / df['floor_count']
    out_cols_dict.update({'floor_ratio': np.float32})

    # 2024-11-14
    df['シェアハウス下宿等'] = 0
    df.loc[df.query('homes_building_name.str.contains("シェアハウス") ' 
                    ' | building_name.str.contains("シェアハウス") '
                    ' | homes_building_name.str.contains("ドーミー")'
                    ' | homes_building_name.str.contains("スチューデント")'
                    ' | building_name.str.contains("スチューデント")').index, 
           'シェアハウス下宿等'] = 1

    df.loc[df['building_id'] == 'a497433', 'シェアハウス下宿等'] = 0
    out_cols_dict.update({'シェアハウス下宿等': pd.Int8Dtype()})


    df = df.join(df_statuses, how='left')
    out_cols_dict.update({col: pd.Int8Dtype() for col in df_statuses.columns})
    # CatBoostのメモリが肥大化して落ちるので、intで処理する。
    # out_cols_dict.update({col: 'category' for col in df_statuses.columns})


    # 森ビルリビングを追加する
    df['MBLiv'] = 0
    df.loc[df['building_name'].isin([
        'a173578','a483385','a177203','a011317','a419598',
        'a015506','a050077','a102943','a013990','a448754',
        'a114716','a157781','a445674','a027818','a065919',
        'a259453', 'a094213'
    ]), 'MBLiv'] = 1
    df['MBLiv'] = df['MBLiv'].astype('category')
    out_cols_dict.update({'MBLiv': 'category'})

    df = df.merge(df_elems, on=['building_id'], how='left')
    df['学校コード'] = df['学校コード'].astype('category')
    out_cols_dict.update({'学校コード': 'category', '小学校_距離': np.float32})

    # 2024-12-01
    df['メゾネットかも'] = 0
    df.loc[
        (df['room_floor'] != df['room_kaisuu']) & 
        (~df['room_floor'].isnull()) & 
        (~df['room_kaisuu'].isnull()),
    'メゾネットかも'] = 1
    out_cols_dict.update({'メゾネットかも': pd.Int8Dtype()})


    if not test:
        # 賃料がおかしいと思われる物件の修正処理
        df.loc[df['unit_id'].isin([36549300, 37558311, 23193245, 
                                   1536099, 3068763, 28639752, 393724,
                                   36922042, # 東京都中央区日本橋小伝馬町14  a365166 ISLAVISTA日本橋
                                   32626608, # 大阪府大阪市西区立売堀1丁目11-15 a072280	ドゥーエ立売堀
                                   22175247, # 福岡県久留米市荘島町17-74 a145150	サンコウハイツ
                                   17360021, # ビジタル浪速5	17360021	大阪府大阪市浪速区恵美須東1丁目2-20
                                   ]), TARGET_COL] = df[TARGET_COL] / 10
        # 専有面積の変更
        df.loc[df.index == 572400, 'house_area'] =  59.40
        df.loc[df['unit_id'] == 4229248, 'house_area'] = 24.37
        df.loc[df['unit_id'] == 22183648, 'house_area'] = 20.20
        df.loc[df['bukken_id'] == 1400180001647, 'house_area'] = 29.70

    if test:
        df.loc[df['index'] == "248028", 'house_area'] = df['unit_area']
        df.loc[df['index'].isin(['025295', '299036']), 'house_area'] = 21.00
        df.loc[df['index'].isin(['161532']), 'house_area'] = 57.0000
        df.loc[df['index'].isin(['006600']), 'house_area'] = 49.6800
        # 224201: house_areaが282.0m^2となっているが1LDKなので、70.559998m^2の間違いではないか?
        df.loc[df['index'].isin(['224201']), 'house_area'] = 70.559998

        # 2024-11-30
        # 200829	202307	a263335	ランドマーク 静岡県三島市梅名101-5	house_area=312.0はおかしいので、78.199997では?	
        df.loc[df['index'] == "200829", 'house_area'] = df['unit_area']
        # 232292	202301	a279060	メゾン・ラフィネ1	埼玉県上尾市大字領家105-1	2023年7月のレコードは、58m^2になっているので、おそらく間違えを修正したのだろう
        df.loc[df['index'] == "232292", 'house_area'] = 58.00
        # 237700	202307	a454636	Coco覚王山	愛知県名古屋市千種区観月町2丁目70	1Kで101.0という広さは考えにくく、25.680000ではないか？
        df.loc[df['index'] == "237700", 'house_area'] = df['unit_area']
        # 70551	202301	a419953	フラワービレッジ  愛知県名古屋市南区三吉町4丁目43-3	187.000000	87.419998 202号室からすると、87m^2では?
        df.loc[df['index'] == "070551", 'house_area'] = 87.00
        # 326377	202201	a326469	セレブYAGA	セレブ・YAGA	193676	東京都目黒区目黒本町3丁目13-22	64.580002	24.360001	共益費からすると24m^2くらいでは？	
        df.loc[df['index'] == "326377", 'house_area'] = df['unit_area']

    return df, out_cols_dict


def dropping_outlier(df: pd.DataFrame) -> pd.DataFrame:
    '''
    '''
    
    df = df.drop([
        125914, 
        125910,
        440988,
        397389,
        203618,
        416163,
        394463,
        127129,
        127125,
        558572,
        300281,
        300280,
        358515,
        78395,
        481129,
        384632,
        381074,
        125913,
        125917,

        # 軽井沢の高いもの 極端に高いものもある(公示地価や他の内容から)
        553332, # ソフィア軽井沢エルミタージュ 1_200_000
        297509, # リベライン軽井沢Aウィング 900_000
        #297508, # リベライン軽井沢Aウィング 266_667

        # 仙台市青葉区パークフラッツ本町
        436880,

        # 成田市のやつ
        412363, # 部屋数などから面積や価格などが不明

        16817, # Dogシャンテ	40134	福岡県福岡市南区横手南町32-1	46.810001

        65743, # a061236	シルクハイツ	青森県青森市青葉2丁目9-6 1m^2

        190586, # コンシェリア新橋 地下1階で店舗用と思われる
    ])

    # * 285800: 1/10となっている？
    # * 107987: 1/10となっている？
    # * 125096: 1/3となっている?
    # * 410743: 1m^2となっている。
    # * 446734: 28.98000m^2と思われる
    # * 43185: 29.70m^2
    # * 165655: 50.000000m^2と思われる
    # * 405859: 25.00m^2と思われる
    # * 146486: 44.00m^2と思われる
    # * 139573: 85.870003m^2と思われる

    # house areaに誤りがあると思われるもの
    drop_targets = df.query(f'{TARGET_COL} >= 3_000_000').index
    df.drop(drop_targets, inplace=True)

    # house areaに誤りがあると思われるもの
    drop_targets = df.query(f'house_area > 250.00 & {TARGET_COL} < 200000').index
    df.drop(drop_targets, inplace=True)

    return df


## Trainデータの生成

In [12]:
df_train, out_cols_dict = preprocessing(df_train, feat_cols_dict)
df_train = dropping_outlier(df_train)

feat_cols = list(set(out_cols_dict.keys()) - set([TARGET_COL]))
cat_cols = [key for key, val in out_cols_dict.items() if val == 'category']

df_train[f'{TARGET_COL}_persqm'] = df_train[TARGET_COL] / df_train['house_area']
df_train = df_train.replace({np.inf: 0, -np.inf: 0})

タグ内容,1階の物件,2階以上,BELS/省エネ基準適合認定建築物,BSアンテナ,CATV,CATV利用料無料,CSアンテナ,IHコンロ,TVモニタ付インターホン,ごみ出し24時間OK,インターネット使用料無料,インターネット対応,ウォークインクローゼット,エアコン,エレベーター,オートバス,オートロック,オール電化,カウンターキッチン,カスタマイズ可,ガスその他,ガスコンロ設置済,ガス暖房,キッズルーム,クローゼット,コンロ一口,コンロ三口,コンロ二口,コンロ四口以上,システムキッチン,シャワー,シャワー付洗面化粧台,シューズインクローゼット,シューズボックス,セキュリティ会社加入済み,タイル貼り,ディスポーザー,デザイナーズ,トイレなし,トランクルーム,バイク置き場あり,バスなし,バス・トイレ別,バリアフリー,バルコニー,フリーアクセス,フリーレント,フロントサービス,フローリング,プロパンガス,ペット不可,ペット可,ペット相談,マンスリー可,メゾネット,ルームシェア不可,ルームシェア可,ロフト付き,下水,事務所不可,事務所可,二世帯住宅向き,二人入居不可,二人入居可,井戸,住宅性能保証制度証明書,保証人不要,保証人要,光ファイバー,免震構造,全居室収納,公営水道,公庫利用可,共同トイレ,共同バス,冷房,冷蔵庫あり,出窓,分譲賃貸,初期費用カード決済可,制震構造,単身者不可,単身者希望,単身者限定,外断熱,太陽光発電システム,女性限定,学生歓迎,学生限定,宅配ボックス,定期借家権,室内洗濯機置場,家具・家電付,家賃カード決済可,家賃・初期費用カード決済可,家賃保証付き,専用トイレ,専用バス,専用庭,床下収納,床暖房,建築条件なし,建築確認完了検査済証,手付金保証あり,排水その他,敷金なし,最上階,有線放送,楽器不可,楽器相談,水道その他,汲取,法人不可,法人希望,法人限定,洗濯機置場あり,洗面所独立,浄化槽,浴室1.6×2.0M以上,浴室TV,浴室乾燥機,温水洗浄便座,満室賃貸中,照明器具付,特優賃（特定優良賃貸住宅）,男性限定,石油暖房,礼金なし,空き家バンク登録物件,給湯,耐震基準適合証明書,耐震構造,角部屋,設計住宅性能評価書,追焚機能,都市ガス,防犯カメラ,電気コンロ,食器洗い乾燥機,駐輪場あり,高温差湯式,高齢者歓迎,高齢者限定
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>
1,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,1,<NA>,1,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>
2,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<N

In [13]:
dispmd(f'### DataFrame Shape = ROWS: {df_train[feat_cols].shape[0]:,}, COLUMNS: {df_train[feat_cols].shape[1]}')
disp(df_train[feat_cols].head(10))

### DataFrame Shape = ROWS: 584,469, COLUMNS: 253

,unit_area_min,dwelling_unit_window_angle,PTD_2020_perN,オートバス,バスなし,最上階,小学校_距離,分譲賃貸,空き家バンク登録物件,病院,ガスコンロ設置済,reform_wet_area_date,money_shuuzen,地価2024,land_setback,building_area_kind,money_sonota1,高齢者歓迎,hospital_distance,IHコンロ,addr1_1,カウンターキッチン,クローゼット,敷金なし,洗面所独立,1階の物件,PTB_2020,ロフト付き,MBLiv,バス・トイレ別,フリーアクセス,シューズボックス,land_youto,下水,ディスポーザー,madori_number_all,家賃・初期費用カード決済可,walk_distance1,トイレなし,PTA_2020,S12_001g,駅までの距離,高齢者限定,flg_investment,park_distance,ペット相談,management_form,家賃カード決済可,電気コンロ,land_toshi,room_floor,駐輪場あり,診療所,カスタマイズ可,ペット不可,unit_area_max,PTC_2020,インターネット対応,専用トイレ,floor_plan_code,キッズルーム,排水その他,madori_kind_all,耐震構造,lon,PTA_2020_perN,bus_time1,二世帯住宅向き,全居室収納,CSアンテナ,BELS/省エネ基準適合認定建築物,室内洗濯機置場,公庫利用可,post,bank_distance,歯科医院,温水洗浄便座,PTE_2020_perN,convenience_distance,太陽光発電システム,礼金なし,バイク置き場あり,year_built,楽器相談,高等学校,法人不可,land_kenpei,CATV利用料無料,オール電化,building_land_chimoku,コンロ二口,保証人不要,浴室乾燥機,BSアンテナ,flg_new,シューズインクローゼット,distance,照明器具付,addr1,二人入居可,usable_status,床下収納,有線放送,PTE_2020,コンロ一口,building_structure,光ファイバー,land_setback_flg,角部屋,land_youseki,女性限定,食器洗い乾燥機,単身者不可,プロパンガス,シャワー付洗面化粧台,保証人要,トランクルーム,冷房,初期費用カード決済可,target_ym_month,PTB_2020_perN,オートロック,専用バス,コンロ三口,水道その他,shopping_street_distance,フリーレント,シャワー,学生限定,都市ガス,usable_date,フローリング,building_area,石油暖房,防犯カメラ,management_association_flg,単身者限定,井戸,PTN_2020,snapshot_window_angle,義務教育学校,耐震基準適合証明書,インターネット使用料無料,デザイナーズ,building_land_area,post1,セキュリティ会社加入済み,床暖房,total_floor_area,法人限定,事務所不可,land_area_all,メゾネットかも,PTC_2020_perN,room_count,reform_exterior_date,ウォークインクローゼット,バルコニー,学生歓迎,バリアフリー,school_jun_distance,フロントサービス,家具・家電付,reform_interior_date,エアコン,PTD_2020,外断熱,住宅性能保証制度証明書,共同バス,学校コード,house_area,empty_number,2階以上,追焚機能,money_sonota2,bus_stop1,給湯,parking_keiyaku,parking_number,男性限定,建築確認完了検査済証,シェアハウス下宿等,ペット可,洗濯機置場あり,ごみ出し24時間OK,設計住宅性能評価書,法人希望,楽器不可,TVモニタ付インターホン,汲取,浄化槽,land_area_kind,ガスその他,ルームシェア可,parking_kubun,免震構造,宅配ボックス,building_status,parking_distance,school_ele_distance,basement_floor_count,出窓,システムキッチン,bukken_type,money_sonota3,super_distance,floor_count,共同トイレ,free_rent_duration,room_kaisuu,target_ym_year,ルームシェア不可,冷蔵庫あり,定期借家権,unit_count,建築条件なし,浴室1.6×2.0M以上,満室賃貸中,building_type,balcony_area,genkyo_code,CATV,コンロ四口以上,単身者希望,money_shuuzenkikin,浴室TV,タイル貼り,land_chisei,家賃保証付き,drugstore_distance,walk_distance2,二人入居不可,高温差湯式,メゾネット,land_road_cond,parking_money,reform_common_area_date,専用庭,建蔽率,特優賃（特定優良賃貸住宅）,容積率,制震構造,地価,幼稚園・こども園,手付金保証あり,lat,エレベーター,公営水道,ガス暖房,マンスリー可,floor_ratio,事務所可,money_kyoueki
0,NaN,5,0.132737,<NA>,<NA>,1,NaN,<NA>,<NA>,9.0,1,<NA>,<NA>,73900.0,0.0,NaN,1760,<NA>,<NA>,<NA>,27,<NA>,1,1,<NA>,<NA>,2643.3328,<NA>,0,1,<NA>,<NA>,NaN,<NA>,<NA>,1,<NA>,1040,<NA>,387.6616,007992,674.504080,<NA>,0,<NA>,<NA>,NaN,<NA>,<NA>,NaN,2,1,44.0,<NA>,<NA>,NaN,1038.4348,<NA>,1,K,<NA>,<NA>,K,<NA>,135.309296,0.095262,0,<NA>,<NA>,<NA>,<NA>,1,<NA>,5980044,<NA>,26.0,<NA>,0.075949,239,<NA>,1,1,213,<NA>,0.0,<NA>,0,<NA>,<NA>,NaN,<NA>,<NA>,1,<NA>,0,<NA>,0.284153,1,27213,<NA>,3,<NA>,<NA>,309.0685,<NA>,1,1,1,<NA>,0,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,7,0.649559,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,202009,1,NaN,<NA>,1,NaN,<NA>,<NA>,4069.4293,5,0.0,<NA>,<NA>,<NA>,NaN,598,<NA>,<NA>,NaN,<NA>,<NA>,NaN,0,0.255179,1,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,540.1642,<NA>,<NA>,<NA>,NaN,23.180000,0.090909,1,<NA>,16500,NaN,1,NaN,1,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,NaN,<NA>,<NA>,1,<NA>,<NA>,1,0,<NA>,<NA>,<NA>,<NA>,3102,26400,1280,2,<NA>,<NA>,2,2020,<NA>,<NA>,<NA>,22,<NA>,<NA>,<NA>,3,2.0,3,1,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,8800,<NA>,<NA>,60.0,<NA>,200.0,<NA>,61583.333333,0.0,<NA>,34.411514,<NA>,<NA>,<NA>,<NA>,1.0,<NA>,3000
1,44.099998,5,0.112648,<NA>,<NA>,<NA>,243.690352,<NA>,<NA>,14.0,<NA>,<NA>,<NA>,513000.0,NaN,NaN,<NA>,<NA>,<NA>,<NA>,13,<NA>,<NA>,<NA>,1,1,14345.8143,<NA>,0,1,<NA>,1,NaN,1,<NA>,2,<NA>,320,<NA>,2079.5352,003294,382.764738,<NA>,0,<NA>,<NA>,NaN,<NA>,<NA>,NaN,1,1,217.0,<NA>,<NA>,44.099998,4344.7174,<NA>,1,DK,<NA>,<NA>,DK,<NA>,139.684555,0.100122,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1730036,<NA>,191.0

## Model

In [ ]:
SEEDS = [2019, 1984]

oversampling = False
fold_strategy = 'KFold'

models = {
    'lgb': [],
}

oof = {}

# LightGBM parameters
params = {
    'lgb': {
        'objective': 'regression',
        'metrics': 'rmse',
        'boosting_type': 'gbdt',
        # 'is_unbalance': True,
        # 'n_estimators': 25000,
        # 'learning_rate': 0.048,
        'n_estimators': 75000,
        'learning_rate': 0.048,

        'extra_trees': True,
        'early_stopping_rounds': 200,

        'bagging_fraction': 0.75,
        'colsample_bynode': 0.75,
        'feature_fraction': 0.75, 

        'verbose': 1
    },

    'cat': {
        'iterations': 30000,
        'eval_metric': 'RMSE',
        'learning_rate': 0.140,
        'early_stopping_rounds': 200,
    }

}


for seed in SEEDS:
    # FOLD
    if fold_strategy == 'Stratified':
       kf = StratifiedKFold(n_splits=FOLD_SIZE - 1, shuffle=True, random_state=seed)
    elif fold_strategy == 'KFold':
        kf = KFold(n_splits=FOLD_SIZE - 1, shuffle=True, random_state=seed)
    else:
        kf = KFold(n_splits=FOLD_SIZE - 1, shuffle=True, random_state=seed)

    gen = kf.split(df_train, df_train[TARGET_COL])

    oof.update({seed: []})
    for num, (train_idx, valid_idx) in enumerate(gen):
        print(f'SEED = {seed} Hold: {num}')
        print(f'train index =', train_idx)
        print(f'valid index =', valid_idx)

        params['lgb'].update({'seed': seed})
        params['cat'].update({'random_seed': seed})
        oof[seed].append(valid_idx)
        
        train_lgb_set = lgb.Dataset(df_train.iloc[train_idx][feat_cols], 
                                    df_train.iloc[train_idx][f'{TARGET_COL}_persqm'], 
                                    categorical_feature=cat_cols)

        eval_lgb_set = lgb.Dataset(df_train.iloc[valid_idx][feat_cols], 
                                    df_train.iloc[valid_idx][f'{TARGET_COL}_persqm'], 
                                    categorical_feature=cat_cols)
                    
        model = lgb.train(params['lgb'], train_lgb_set, 
                          valid_sets=[train_lgb_set, 
                                      eval_lgb_set],
                          categorical_feature=cat_cols,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=True),
                              lgb.log_evaluation(1000)
                          ])
        
        pickle.dump(model, open(fr'{MODEL_PATH}/model1_feats_{len(model.feature_name())}_lgb_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'wb'))        
        del model
        del train_lgb_set
        del eval_lgb_set
        gc.collect()

        # # CatBoost
        X_tr = copy.copy(df_train.iloc[train_idx][feat_cols])
        X_tr[cat_cols] = X_tr[cat_cols].astype(str)

        X_vl = copy.copy(df_train.iloc[valid_idx][feat_cols])
        X_vl[cat_cols] = X_vl[cat_cols].astype(str)
        
        train_cat_pool = cat.Pool(X_tr, 
                                  df_train.iloc[train_idx][f'{TARGET_COL}_persqm'],
                                  cat_features=cat_cols)
        
        eval_cat_pool = cat.Pool(X_vl, 
                                 df_train.iloc[valid_idx][f'{TARGET_COL}_persqm'], 
                                 cat_features=cat_cols)        
                
        model = cat.CatBoostRegressor(**params['cat'])
        model.fit(train_cat_pool, 
                  eval_set=eval_cat_pool, verbose=100)

        # models['cat'].append(model)
        pickle.dump(model, open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_cat_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'wb'))
        del X_tr
        del X_vl
        del model
        del train_cat_pool
        del eval_cat_pool
        gc.collect()


SEED = 2019 Hold: 0
train index = [     0      2      3 ... 584464 584466 584468]
valid index = [     1      5      7 ... 584459 584465 584467]
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077087 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 48790
[LightGBM] [Info] Number of data points in the train set: 438351, number of used features: 246
[LightGBM] [Info] Start training from score 1743.752815
Training until validation scores don't improve for 200 rounds
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 173.784	valid_1's rmse: 188.044
[2000]	training's rmse: 160.418	v

In [16]:
from sklearn.metrics import root_mean_squared_error

df_train['predict_lgb_2019'] = 0
df_train['predict_cat_2019'] = 0
df_train['predict_lgb_1984'] = 0
df_train['predict_cat_1984'] = 0
# df_train['predict_cat'] = 0

i = 0
for seed in SEEDS:
    for num, valid_idx in enumerate(oof[seed]):
        mdl = pickle.load(open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_lgb_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'rb'))
        pred = mdl.predict(df_train.iloc[valid_idx][feat_cols])
        
        score_1 = root_mean_squared_error(df_train.iloc[valid_idx][f'{TARGET_COL}_persqm'], pred)
        score_2 = root_mean_squared_error(df_train.iloc[valid_idx][TARGET_COL], pred * df_train.iloc[valid_idx]['house_area'])

        df_train.iloc[valid_idx, -4 + 2 * i] = pred * df_train.iloc[valid_idx]['house_area']

        dispmd('## LightGBM')
        dispmd(f'#{num}: RMSE of house area per sqm = {score_1}')    
        dispmd(f'#{num}: RMSE = {score_2}')


        X_vl = copy.copy(df_train.iloc[valid_idx][feat_cols])
        X_vl[cat_cols] = X_vl[cat_cols].astype(str)
        val_cat_pool = cat.Pool(X_vl, cat_features=cat_cols)

        mdl = pickle.load(open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_cat_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'rb'))
        pred = mdl.predict(val_cat_pool)

        score_1 = root_mean_squared_error(df_train.iloc[valid_idx][f'{TARGET_COL}_persqm'], pred)
        score_2 = root_mean_squared_error(df_train.iloc[valid_idx][TARGET_COL], pred * df_train.iloc[valid_idx]['house_area'])

        df_train.iloc[valid_idx, -3 + 2 * i] = pred * df_train.iloc[valid_idx]['house_area']

        dispmd('## CatBoost')
        dispmd(f'#{num}: RMSE of house area per sqm = {score_1}')    
        dispmd(f'#{num}: RMSE = {score_2}')

    i += 1


del mdl
gc.collect()    

## LightGBM

#0: RMSE of house area per sqm = 167.34730370499653

#0: RMSE = 6460.01923368339

## CatBoost

#0: RMSE of house area per sqm = 168.5021711576499

...

#3: RMSE = 6523.793706666718

## CatBoost

#3: RMSE of house area per sqm = 169.34258907226135

#3: RMSE = 6901.584326888702

0

In [17]:
df_ly2 = df_train[['bukken_id', TARGET_COL, 'predict_lgb_2019', 'predict_lgb_1984', 'predict_cat_2019', 'predict_cat_1984']]
df_ly2.to_csv('layer2_output.csv')

## CVスコアの確認


In [18]:
score_1 = root_mean_squared_error(df_train[TARGET_COL], 
                                 (df_train['predict_lgb_2019'] + df_train['predict_lgb_1984']) / 2)
dispmd(f'### LightGBM: CV Score = {score_1}')

score_1 = root_mean_squared_error(df_train[TARGET_COL], 
                                (df_train['predict_cat_2019'] + df_train['predict_cat_1984']) / 2)
dispmd(f'### CatBoost: CV Score = {score_1}')

score_1 = root_mean_squared_error(df_train[TARGET_COL], 
                                  (df_train['predict_lgb_2019'] + df_train['predict_lgb_1984'] + df_train['predict_cat_2019'] + df_train['predict_cat_1984']) / 4)
dispmd(f'### Mean Model: CV Score = {score_1}')

# del df_train
# gc.collect()

### LightGBM: CV Score = 6422.876304980938

### CatBoost: CV Score = 6502.13457233743

### Mean Model: CV Score = 6320.169589299979

In [19]:
SPLIT_SIZE = 4
SEED = 2021

ly2_feat_cols = [
    'predict_lgb_2019',
    'predict_lgb_1984',
    'predict_cat_2019',
    'predict_cat_1984'
]

models = []
kf = KFold(n_splits=SPLIT_SIZE, shuffle=True, random_state=SEED)
gen = kf.split(df_ly2)

params = {
        'objective': 'reg:squarederror',
        'booster': 'gblinear',
        'eta': 0.03,
        'eval_metric': 'rmse',
        'seed': SEED,
}

for num, (train_idx, valid_idx) in enumerate(gen):
    X1_tr, X1_vl  = df_ly2.iloc[train_idx][ly2_feat_cols], df_ly2.iloc[valid_idx][ly2_feat_cols]
    Y_tr, Y_vl    = df_ly2.iloc[train_idx][TARGET_COL], df_ly2.iloc[valid_idx][TARGET_COL]

    train_set = xgb.DMatrix(X1_tr, Y_tr, feature_names=ly2_feat_cols)
    valid_set = xgb.DMatrix(X1_vl, Y_vl, feature_names=ly2_feat_cols)

    model = xgb.train(
             params=params, 
             dtrain=train_set,
             evals=[
                 (train_set, 'train'),
                 (valid_set, 'valid')
             ],
             num_boost_round=10000, 
             early_stopping_rounds=200,
            #  eval_metric='rmse',
            #  maximize=False,
             verbose_eval=50,
         )
    
    models.append(model)
    

[0]	train-rmse:30932.10011	valid-rmse:30248.55389
[50]	train-rmse:22311.33229	valid-rmse:22004.90968
[100]	train-rmse:17284.07582	valid-rmse:17107.66101
[150]	train-rmse:13642.80116	valid-rmse:13560.02704
[200]	train-rmse:11068.91959	valid-rmse:11049.99625
[250]	train-rmse:9307.55255	valid-rmse:9327.65354
[300]	train-rmse:8145.60313	valid-rmse:8184.93783
[350]	train-rmse:7408.09131	valid-rmse:7452.36656
[400]	train-rmse:6956.41845	valid-rmse:6996.86192
[450]	train-rmse:6688.40675	valid-rmse:6720.87319
[500]	train-rmse:6531.93526	valid-rmse:6555.24156
[550]	train-rmse:6441.56405	valid-rmse:6456.13166
[600]	train-rmse:6389.90951	valid-rmse:6396.91777
[650]	train-rmse:6360.45264	valid-rmse:6361.21262
[700]	train-rmse:6343.73038	valid-rmse:6339.51396
[750]	train-rmse:6334.21674	valid-rmse:6326.09126
[800]	train-rmse:6328.80785	valid-rmse:6317.66932
[850]	train-rmse:6325.72893	valid-rmse:6312.29665
[900]	train-rmse:6323.96814	valid-rmse:6308.80613
[950]	train-rmse:6322.95007	valid-rmse:6306

In [20]:
for seed in SEEDS:
    feats, gains, splits = [], [], []
    for num in range(FOLD_SIZE - 1):
        mdl = pickle.load(open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_lgb_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'rb'))

        feats.append(mdl.feature_name())
        gains.append(mdl.feature_importance(importance_type='gain'))
        splits.append(mdl.feature_importance(importance_type='split'))

    # with pd.ExcelWriter('feature_importance.xlsx', mode='w') as writer:
            # fi_df.to_excel(writer, sheet_name=f'CV_{num}')
    for num, (feat, gain, split) in enumerate(zip(feats, gains, splits), 1):
        fi_df = pd.DataFrame()
        fi_df['features'] = feat
        fi_df['importance_gain'] = 0
        fi_df['importance_split'] = 0

        fi_df['importance_gain'] = gain
        fi_df['importance_split'] = split
        fi_df.sort_values(['importance_gain'], ascending=False, inplace=True)

        dispmd(f'### #{num}')
        disp(fi_df.head(30))
        disp(fi_df.query('importance_split == 0'))
        print(fi_df.query('importance_split == 0')['features'].to_list())

### #1

,features,importance_gain,importance_split
98,addr1,6.018036e+11,22122
13,地価2024,3.888540e+11,9774
20,addr1_1,2.667384e+11,25459
145,post1,2.242118e+11,43949
75,歯科医院,2.066890e+11,5518
170,house_area,1.515000e+11,23260
62,madori_kind_all,1.283579e+11,11258
138,PTN_2020,1.025952e+11,5912
82,year_built,9.387758e+10,17706
35,madori_number_all,8.498113e+10,2200


,features,importance_gain,importance_split
119,target_ym_month,0.0,0
70,BELS/省エネ基準適合認定建築物,0.0,0
217,満室賃貸中,0.0,0
28,MBLiv,0.0,0
185,設計住宅性能評価書,0.0,0
180,建築確認完了検査済証,0.0,0
167,住宅性能保証制度証明書,0.0,0
141,耐震基準適合証明書,0.0,0
72,公庫利用可,0.0,0


['target_ym_month', 'BELS/省エネ基準適合認定建築物', '満室賃貸中', 'MBLiv', '設計住宅性能評価書', '建築確認完了検査済証', '住宅性能保証制度証明書', '耐震基準適合証明書', '公庫利用可']


### #2

...

['BELS/省エネ基準適合認定建築物', 'target_ym_month', '設計住宅性能評価書', '満室賃貸中', '建築確認完了検査済証', '耐震基準適合証明書', '住宅性能保証制度証明書', '公庫利用可', 'MBLiv']


### #4

,features,importance_gain,importance_split
98,addr1,5.280137e+11,19443
13,地価2024,4.264323e+11,8596
20,addr1_1,3.109617e+11,22656
52,診療所,2.114293e+11,5268
145,post1,1.994711e+11,39728
170,house_area,1.655433e+11,20777
62,madori_kind_all,1.440128e+11,10067
138,PTN_2020,1.275931e+11,5294
82,year_built,1.113410e+11,16227
59,floor_plan_code,9.381360e+10,10576


,features,importance_gain,importance_split
141,耐震基準適合証明書,0.0,0
119,target_ym_month,0.0,0
70,BELS/省エネ基準適合認定建築物,0.0,0
217,満室賃貸中,0.0,0
185,設計住宅性能評価書,0.0,0
28,MBLiv,0.0,0
72,公庫利用可,0.0,0
167,住宅性能保証制度証明書,0.0,0
180,建築確認完了検査済証,0.0,0


['耐震基準適合証明書', 'target_ym_month', 'BELS/省エネ基準適合認定建築物', '満室賃貸中', '設計住宅性能評価書', 'MBLiv', '公庫利用可', '住宅性能保証制度証明書', '建築確認完了検査済証']


## 三井不動産 パークマンション
* 三田綱町パークマンション(a005948)
* パークマンション千鳥ヶ淵(a419423)	
* 麻布霞町パークマンション(a531772)
* 縮景園パークマンション(a485959)

## 森ビル レジデンス
* オランダヒルズ森タワー(a173578): 3, 0
* アークヒルズ仙石山(a483385): 1, 2
* 六本木ヒルズレジデンスA棟(a177203): 1, 3
* 六本木ヒルズレジデンスB棟(a011317): 3, 4
* 六本木ヒルズレジデンスC棟(a419598): 1, 1
* 六本木ヒルズレジデンスD棟(a015506): 3, 1
* アークタワーズイースト(a050077): 1, 2
* 元麻布ヒルズフォレストタワー(a102943): 0, 1
* 愛宕グリーンヒルズフォレストタワー(a013990): 3, 0
* 六本木ヒルズゲートタワー(a448754): 3, 2
* 六本木さくら坂レジデンス(a114716): 1, 0
* フォレストテラス鳥居坂(a157781): 2, 0
* 赤坂タワーレジデンストップオブザヒル(a445674): 3, 4
* アークフォレストテラス(a027818): 1, 0
* 六本木ファーストプラザ(a065919): 4, 2
* アークヒルズフロントタワー(a259453): 2, 0
* アークヒルズエグゼクティブタワー(a094213): 0, 3

## 住友不動産
* a061776	グランドヒルズ目黒東ヶ丘
* a125678	グランドヒルズ恵比寿
* a091990	グランドヒルズ一番町
* a497198	グランドヒルズ覚王山法王町松風の邸
* a537310	等々力グランドヒルズ
* a022520	小日向グランドヒルズ


* 麻布台ヒルズレジデンス: 0, 0
* 虎ノ門ヒルズ: 0, 0
* The Upper House: 0, 0
* 赤坂溜池タワーレジデンス: 0, 0
* フォレストテラス松濤: 0, 0

* オランダヒルズ森タワー(a173578): 3, 0
* アークヒルズ仙石山(a483385): 1, 2
* 六本木ヒルズレジデンスA棟(a177203): 1, 3
* 六本木ヒルズレジデンスB棟(a011317): 3, 4
* 六本木ヒルズレジデンスC棟(a419598): 1, 1
* 六本木ヒルズレジデンスD棟(a015506): 3, 1
* アークタワーズイースト(a050077): 1, 2
* 元麻布ヒルズフォレストタワー(a102943): 0, 1
* 愛宕グリーンヒルズフォレストタワー(a013990): 3, 0
* 六本木ヒルズゲートタワー(a448754): 3, 2
* 六本木さくら坂レジデンス(a114716): 1, 0
* フォレストテラス鳥居坂(a157781): 2, 0
* 赤坂タワーレジデンストップオブザヒル(a445674): 3, 4
* アークフォレストテラス(a027818): 1, 0
* 六本木ファーストプラザ(a065919): 4, 2
* アークヒルズフロントタワー(a259453): 2, 0
* アークヒルズエグゼクティブタワー(a094213): 0, 3

drop cols =
['高齢者限定', 'カスタマイズ可', '法人限定', 'シェアハウス下宿等', '特優賃（特定優良賃貸住宅）', 'フリーアクセス', '建築条件なし', 'BELS/省エネ基準適合認定建築物', 'トイレなし', '幼稚園・こども園', '建築確認完了検査済証', '汲取', '公庫利用可', 'addr1_1', 'money_shuuzenkikin', '空き家バンク登録物件', '住宅性能保証制度証明書', '満室賃貸中', '設計住宅性能評価書', '二世帯住宅向き', '耐震基準適合証明書', '太陽光発電システム', '井戸', '高等学校', 'target_ym_month', 'money_shuuzen', '手付金保証あり', '石油暖房']


```plain
#0: RMSE of house area per sqm = 532.1715852761931
#0: RMSE = 47132.32488323217
#1: RMSE of house area per sqm = 492.8414270074763
#1: RMSE = 46755.23268745363
#2: RMSE of house area per sqm = 461.7416063839124
#2: RMSE = 47546.60973974178
#3: RMSE of house area per sqm = 552.380619952654
#3: RMSE = 56330.09965018029
```

## Submission

In [21]:
path = './test.csv.gz'

feat_cols_dict.update({'index': str})
read_opts = {
    'encoding': 'utf8', 
    'dtype': feat_cols_dict,
    'parse_dates': [
        'building_create_date',
        'building_modify_date',
        'reform_date',
        'renovation_date',
        'snapshot_create_date',
        'new_date',
        'snapshot_modify_date'
    ]
}

df_test = pd.read_csv(path, **read_opts)
dispmd(f'### DataFrame Shape = ROWS: {df_test.shape[0]:,}, COLUMNS: {df_test.shape[1]}')
disp(df_test.head())

df_test['building_name'] = df_test['building_name'].fillna('(None)')

# df_test, _, _ = preprocessing(df_test, feat_cols_dict, test=True, df_train_agg=df_train_agg)
df_test, _ = preprocessing(df_test, feat_cols_dict, test=True)
dispmd(f'### DataFrame Shape = ROWS: {df_test.shape[0]:,}, COLUMNS: {df_test.shape[1]}')
disp(df_test[feat_cols].head(10))

### DataFrame Shape = ROWS: 384,540, COLUMNS: 152

,index,target_ym,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,homes_building_name_ruby,unit_count,full_address,lon,lat,building_structure,total_floor_area,building_area,floor_count,basement_floor_count,year_built,building_land_area,land_area_all,unit_area_min,unit_area_max,building_land_chimoku,land_youto,land_toshi,land_chisei,land_area_kind,land_setback_flg,land_setback,land_kenpei,land_youseki,land_road_cond,land_seigen,building_area_kind,management_form,management_association_flg,reform_exterior,reform_exterior_other,reform_exterior_date,reform_common_area,reform_common_area_date,building_tag_id,unit_id,unit_name,name_ruby,room_floor,balcony_area,dwelling_unit_window_angle,room_count,unit_area,floor_plan_code,reform_date,reform_place,reform_place_other,reform_wet_area,reform_wet_area_other,reform_wet_area_date,reform_interior,reform_interior_other,reform_interior_date,reform_etc,renovation_date,renovation_etc,unit_tag_id,bukken_id,snapshot_create_date,new_date,snapshot_modify_date,timelimit_date,flg_open,flg_own,bukken_type,flg_investment,empty_number,empty_contents,post1,post2,addr1_1,addr1_2,addr2_name,addr3_name,addr4_name,nl,el,rosen_name1,eki_name1,bus_stop1,bus_time1,walk_distance1,rosen_name2,eki_name2,bus_stop2,bus_time2,walk_distance2,traffic_other,traffic_car,snapshot_land_area,snapshot_land_shidou,land_shidou_a,land_shidou_b,land_mochibun_a,land_mochibun_b,house_area,flg_new,house_kanrinin,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,money_kyoueki,money_kyoueki_tax,money_rimawari_now,money_shuuzen,money_shuuzenkikin,money_sonota_str1,money_sonota1,money_sonota_str2,money_sonota2,money_sonota_str3,money_sonota3,parking_money,parking_money_tax,parking_kubun,parking_distance,parking_number,parking_memo,genkyo_code,usable_status,usable_date,school_ele_name,school_ele_distance,school_ele_code,school_jun_name,school_jun_distance,school_jun_code,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,000000,202207,a372446,1,2014-06-27 20:30:14,2024-02-09 00:10:05,3,シャーメゾンエランA,NaN,シャーメゾンELAN A棟,シャーメゾンエラン Aトウ,6,山口県山陽小野田市大字西高泊618-12,131.175430,34.007275,10,NaN,NaN,2,<NA>,201101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,210301/321001/210101/210401/320901,23270592,202,NaN,2,NaN,6,2,62.509998,250,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaT,NaN,340201/230103/310501/230201/230501/220501/3401...,37220460000756,2022-04-18,2022-04-18,2022-07-25 14:00:26,2022-08-01 00:00:00,1,1,3102,0,<NA>,A0202,756,57,35,216,大字西高泊,618-12,NaN,122414490.0,472240000.0,JR山陽本線,小野田,高須,4,160,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.000000,0,NaN,2,5,2,50,3000,3,NaN,<NA>,<NA>,鍵販売手数料,11000,安心サポート24プラス利用料月額,770,町内会費月額,500,<NA>,2,1,<NA>,<NA>,1台無料 地上,2,1,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,110102/110903/121002/210301/220101/220201/2203...,1,【個人契約】 初回契約事務手数料：22，000円（税込）、月額保証料：賃料等の1％,<NA>,NaN
1,000001,202301,a276097,1,2014-06-28 02:47:45,2024-08-15 02:24:34,1,マルコフォート,NaN,マルコフォート,マルコフオート,15,東京都調布市菊野台3丁目3-3,139.569168,35.653206,4,NaN,NaN,3,1,198703,NaN,NaN,18.000000,20.000000,NaN,NaN,NaN,NaN,NaN,1,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,210101/321001/320901/210301/210201,853089,303,NaN,3,NaN,5,1,19.440001,120,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaT,NaN,230801/240104/310501/263101/290101/230201/2605...,1038270025409,2023-01-30,2023-01-30,2023-01-30 16:44:45,2023-02-06 00:00:00,1,1,3101,0,2,303,182,7,13,208,菊野台３丁目,NaN,3-3,128339891.0,502460485.0,京王線,柴崎,NaN,<NA>,240,京王線,つつじヶ丘,NaN,NaN,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,0,NaN,3,5,1,20,2000,3,NaN,<NA>,<NA>,消毒料,16500,入居安心サービス料,16500,NaN,<NA>,0,NaN,4,<NA>,<NA>,NaN,3,3,202303,市立調和小

タグ内容,1階の物件,2階以上,BELS/省エネ基準適合認定建築物,BSアンテナ,CATV,CATV利用料無料,CSアンテナ,IHコンロ,TVモニタ付インターホン,ごみ出し24時間OK,インターネット使用料無料,インターネット対応,ウォークインクローゼット,エアコン,エレベーター,オートバス,オートロック,オール電化,カウンターキッチン,カスタマイズ可,ガスその他,ガスコンロ設置済,ガス暖房,キッズルーム,クローゼット,コンロ一口,コンロ三口,コンロ二口,コンロ四口以上,システムキッチン,シャワー,シャワー付洗面化粧台,シューズインクローゼット,シューズボックス,セキュリティ会社加入済み,タイル貼り,テラス,ディスポーザー,デザイナーズ,トイレなし,トランクルーム,バイク置き場あり,バスなし,バス・トイレ別,バリアフリー,バルコニー,パントリー,フリーアクセス,フリーレント,フロントサービス,フローリング,プロパンガス,ペット不可,ペット可,ペット相談,マンスリー可,メゾネット,ルーフバルコニー,ルームシェア不可,ルームシェア可,ロフト付き,下水,事務所不可,事務所可,二世帯住宅向き,二人入居不可,二人入居可,井戸,住宅性能保証制度証明書,保証人不要,保証人要,光ファイバー,免震構造,全居室収納,公営水道,公庫利用可,共同トイレ,共同バス,冷房,冷蔵庫あり,出窓,分譲賃貸,初期費用カード決済可,制震構造,単身者不可,単身者希望,単身者限定,吹き抜け,外断熱,太陽光発電システム,女性限定,学生歓迎,学生限定,宅配ボックス,定期借家権,室内洗濯機置場,家具・家電付,家賃カード決済可,家賃・初期費用カード決済可,家賃保証付き,専用トイレ,専用バス,専用庭,床下収納,床暖房,建築条件なし,建築確認完了検査済証,建設住宅性能評価書（新築時）,建設住宅性能評価書（既存住宅）,手付金保証あり,排水その他,敷金なし,最上階,有線放送,楽器不可,楽器相談,水道その他,汲取,法人不可,法人希望,法人限定,洗濯機置場あり,洗面所独立,浄化槽,浄水器・活水器,浴室1.6×2.0M以上,浴室TV,浴室乾燥機,浴室暖房,温水洗浄便座,満室賃貸中,照明器具付,特優賃（特定優良賃貸住宅）,男性限定,石油暖房,礼金なし,空き家バンク登録物件,給湯,耐震基準適合証明書,耐震構造,角部屋,設計住宅性能評価書,追焚機能,都市ガス,防犯カメラ,電気コンロ,食器洗い乾燥機,駐輪場あり,高温差湯式,高齢者歓迎,高齢者限定
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>
1,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>
2,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>
3,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>
4,<NA>,1,<NA>,<NA>,<NA>,<

### DataFrame Shape = ROWS: 384,540, COLUMNS: 341

,unit_area_min,dwelling_unit_window_angle,PTD_2020_perN,オートバス,バスなし,最上階,小学校_距離,分譲賃貸,空き家バンク登録物件,病院,ガスコンロ設置済,reform_wet_area_date,money_shuuzen,地価2024,land_setback,building_area_kind,money_sonota1,高齢者歓迎,hospital_distance,IHコンロ,addr1_1,カウンターキッチン,クローゼット,敷金なし,洗面所独立,1階の物件,PTB_2020,ロフト付き,MBLiv,バス・トイレ別,フリーアクセス,シューズボックス,land_youto,下水,ディスポーザー,madori_number_all,家賃・初期費用カード決済可,walk_distance1,トイレなし,PTA_2020,S12_001g,駅までの距離,高齢者限定,flg_investment,park_distance,ペット相談,management_form,家賃カード決済可,電気コンロ,land_toshi,room_floor,駐輪場あり,診療所,カスタマイズ可,ペット不可,unit_area_max,PTC_2020,インターネット対応,専用トイレ,floor_plan_code,キッズルーム,排水その他,madori_kind_all,耐震構造,lon,PTA_2020_perN,bus_time1,二世帯住宅向き,全居室収納,CSアンテナ,BELS/省エネ基準適合認定建築物,室内洗濯機置場,公庫利用可,post,bank_distance,歯科医院,温水洗浄便座,PTE_2020_perN,convenience_distance,太陽光発電システム,礼金なし,バイク置き場あり,year_built,楽器相談,高等学校,法人不可,land_kenpei,CATV利用料無料,オール電化,building_land_chimoku,コンロ二口,保証人不要,浴室乾燥機,BSアンテナ,flg_new,シューズインクローゼット,distance,照明器具付,addr1,二人入居可,usable_status,床下収納,有線放送,PTE_2020,コンロ一口,building_structure,光ファイバー,land_setback_flg,角部屋,land_youseki,女性限定,食器洗い乾燥機,単身者不可,プロパンガス,シャワー付洗面化粧台,保証人要,トランクルーム,冷房,初期費用カード決済可,target_ym_month,PTB_2020_perN,オートロック,専用バス,コンロ三口,水道その他,shopping_street_distance,フリーレント,シャワー,学生限定,都市ガス,usable_date,フローリング,building_area,石油暖房,防犯カメラ,management_association_flg,単身者限定,井戸,PTN_2020,snapshot_window_angle,義務教育学校,耐震基準適合証明書,インターネット使用料無料,デザイナーズ,building_land_area,post1,セキュリティ会社加入済み,床暖房,total_floor_area,法人限定,事務所不可,land_area_all,メゾネットかも,PTC_2020_perN,room_count,reform_exterior_date,ウォークインクローゼット,バルコニー,学生歓迎,バリアフリー,school_jun_distance,フロントサービス,家具・家電付,reform_interior_date,エアコン,PTD_2020,外断熱,住宅性能保証制度証明書,共同バス,学校コード,house_area,empty_number,2階以上,追焚機能,money_sonota2,bus_stop1,給湯,parking_keiyaku,parking_number,男性限定,建築確認完了検査済証,シェアハウス下宿等,ペット可,洗濯機置場あり,ごみ出し24時間OK,設計住宅性能評価書,法人希望,楽器不可,TVモニタ付インターホン,汲取,浄化槽,land_area_kind,ガスその他,ルームシェア可,parking_kubun,免震構造,宅配ボックス,building_status,parking_distance,school_ele_distance,basement_floor_count,出窓,システムキッチン,bukken_type,money_sonota3,super_distance,floor_count,共同トイレ,free_rent_duration,room_kaisuu,target_ym_year,ルームシェア不可,冷蔵庫あり,定期借家権,unit_count,建築条件なし,浴室1.6×2.0M以上,満室賃貸中,building_type,balcony_area,genkyo_code,CATV,コンロ四口以上,単身者希望,money_shuuzenkikin,浴室TV,タイル貼り,land_chisei,家賃保証付き,drugstore_distance,walk_distance2,二人入居不可,高温差湯式,メゾネット,land_road_cond,parking_money,reform_common_area_date,専用庭,建蔽率,特優賃（特定優良賃貸住宅）,容積率,制震構造,地価,幼稚園・こども園,手付金保証あり,lat,エレベーター,公営水道,ガス暖房,マンスリー可,floor_ratio,事務所可,money_kyoueki
0,NaN,6,0.138470,<NA>,<NA>,1,NaN,<NA>,<NA>,1.0,<NA>,<NA>,<NA>,46900.0,NaN,NaN,11000,<NA>,<NA>,1,35,<NA>,<NA>,<NA>,<NA>,<NA>,1447.3136,<NA>,0,1,<NA>,<NA>,NaN,1,<NA>,2,<NA>,160,<NA>,350.3596,008575,823.648193,<NA>,0,<NA>,<NA>,NaN,<NA>,<NA>,NaN,2,1,22.0,<NA>,1,NaN,657.8255,<NA>,1,LDK,<NA>,<NA>,LDK,<NA>,131.175430,0.142684,4,<NA>,<NA>,<NA>,<NA>,1,<NA>,7560057,<NA>,11.0,1,0.092166,<NA>,<NA>,<NA>,<NA>,138,<NA>,0.0,<NA>,<NA>,<NA>,1,NaN,<NA>,1,<NA>,<NA>,0,<NA>,0.449472,<NA>,35216,<NA>,1,<NA>,<NA>,226.3142,1,10,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,7,0.589417,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,NaN,<NA>,<NA>,NaN,<NA>,<NA>,2455.4987,5,0.0,<NA>,<NA>,<NA>,NaN,756,<NA>,<NA>,NaN,<NA>,<NA>,NaN,0,0.267899,2,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,340.0125,<NA>,<NA>,<NA>,NaN,62.000000,<NA>,1,<NA>,770,高須,1,1,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,NaN,<NA>,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,3102,500,<NA>,2,<NA>,<NA>,2,2022,<NA>,<NA>,<NA>,6,<NA>,<NA>,<NA>,3,NaN,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,60.0,<NA>,200.0,<NA>,3.908333e+04,0.0,<NA>,34.007275,<NA>,<NA>,<NA>,<NA>,1.0,<NA>,3000
1,18.000000,5,0.107811,<NA>,<NA>,1,434.809410,<NA>,<NA>,3.0,<NA>,<NA>,<NA>,393000.0,NaN,NaN,16500,<NA>,<NA>,<NA>,13,<NA>,<NA>,<NA>,<NA>,<NA>,10086.7876,<NA>,0,<NA>,<NA>,1,NaN,<NA>,<NA>,1,<NA>,240,<NA>,1599.8343,003959,205.956666,<NA>,0,<NA>,<NA>,NaN,<NA>,1,NaN,3,<NA>,151.0,<NA>,<NA>,20.000000,2934.3573,<NA>,<NA>,K,<NA>,<NA>,K,<NA>,139.569168,0.109420,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

## LightGBMとCatBoost

In [22]:
df_pred = df_test[['index']]

# CatBoost
X_te = copy.copy(df_test[feat_cols])
X_te[cat_cols] = X_te[cat_cols].astype(str)

test_cat_pool = cat.Pool(X_te, cat_features=cat_cols)

for seed in SEEDS:
    df_pred[f'predict_lgb_{seed}'] = 0
    df_pred[f'predict_cat_{seed}'] = 0
    
    for num in range(FOLD_SIZE - 1):
        
        mdl = pickle.load(open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_lgb_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'rb'))
        df_pred[f'predict_lgb_{seed}'] += mdl.predict(df_test[feat_cols]) / (FOLD_SIZE - 1)

        mdl = pickle.load(open(fr'{MODEL_PATH}/model1_feats_{len(feat_cols)}_cat_fold_{num}of{FOLD_SIZE}_seed_{seed}.pkl', 'rb'))
        df_pred[f'predict_cat_{seed}'] += mdl.predict(test_cat_pool) / (FOLD_SIZE - 1)

        dispmd(f'### #{num} Models Output')
        disp(df_pred.head())


for seed in SEEDS:
    df_pred[f'predict_lgb_{seed}'] *= df_test['house_area']
    df_pred[f'predict_cat_{seed}'] *= df_test['house_area']
disp(df_pred.head(10))

### #0 Models Output

,index,predict_lgb_2019,predict_cat_2019
0,000000,278.473677,273.850806
1,000001,651.663906,688.147442
2,000002,251.537947,253.278731
3,000003,259.378084,257.925633
4,000004,886.816181,916.815180


### #1 Models Output

,index,predict_lgb_2019,predict_cat_2019
0,000000,556.514655,538.964591
1,000001,1279.057000,1346.265581
2,000002,501.024407,518.942673
3,000003,514.953223,522.736778
4,000004,1796.620703,1848.842800


### #2 Models Output

...

### #2 Models Output

,index,predict_lgb_2019,predict_cat_2019,predict_lgb_1984,predict_cat_1984
0,000000,1070.217249,1081.970153,796.404706,797.447274
1,000001,2573.947042,2709.771922,1926.813479,2004.140586
2,000002,1002.026665,1035.851291,719.795857,774.249722
3,000003,1026.393262,1049.784339,758.936034,791.215505
4,000004,3561.313088,3604.629538,2592.256024,2640.483707


### #3 Models Output

,index,predict_lgb_2019,predict_cat_2019,predict_lgb_1984,predict_cat_1984
0,000000,1070.217249,1081.970153,1052.864741,1068.921108
1,000001,2573.947042,2709.771922,2573.245168,2705.283201
2,000002,1002.026665,1035.851291,967.736088,1031.790196
3,000003,1026.393262,1049.784339,1007.843624,1044.537697
4,000004,3561.313088,3604.629538,3484.813168,3515.181296


,index,predict_lgb_2019,predict_cat_2019,predict_lgb_1984,predict_cat_1984
0,000000,66353.469432,67082.149468,65277.613958,66273.108700
1,000001,48904.993797,51485.666510,48891.658193,51400.380814
2,000002,61123.626548,63186.928742,59031.901382,62939.201934
3,000003,47214.090031,48290.079574,46360.806681,48048.734072
4,000004,80877.416967,81861.133504,79140.103846,79829.764021
5,000005,34612.378884,37862.775470,36083.912141,36904.993774
6,000006,89602.604049,91455.010143,90865.507706,92263.179575
7,000007,50504.610054,53087.618556,50595.346917,52502.538939
8,000008,64983.259189,63962.524605,65745.354942,64361.126331
9,000009,67768.413052,65642.277494,67144.742076,65179.928797


In [23]:
df_pred = df_pred[[
    'index',
    'predict_lgb_2019', 'predict_lgb_1984', 
    'predict_cat_2019', 'predict_cat_1984'
]]

test_set = xgb.DMatrix(df_pred[df_pred.columns[df_pred.columns.str.contains('predict')]])

for num, mdl in enumerate(models):
    df_pred[f'predict_ly2_{num}'] = mdl.predict(test_set)

df_pred['predict'] = df_pred[df_pred.columns[df_pred.columns.str.contains('predict_ly2')]].mean(axis=1)
df_pred['predict'] = df_pred['predict'].astype(int)
disp(df_pred.head(10))

,index,predict_lgb_2019,predict_lgb_1984,predict_cat_2019,predict_cat_1984,predict_ly2_0,predict_ly2_1,predict_ly2_2,predict_ly2_3,predict
0,000000,66353.469432,65277.613958,67082.149468,66273.108700,66283.406250,66337.367188,66289.851562,66300.359375,66302
1,000001,48904.993797,48891.658193,51485.666510,51400.380814,50108.445312,50143.226562,50116.316406,50060.031250,50107
2,000002,61123.626548,59031.901382,63186.928742,62939.201934,61563.875000,61622.210938,61586.625000,61546.765625,61579
3,000003,47214.090031,46360.806681,48290.079574,48048.734072,47416.265625,47439.554688,47404.750000,47381.824219,47410
4,000004,80877.416967,79140.103846,81861.133504,79829.764021,80536.476562,80614.593750,80558.406250,80595.109375,80576
5,000005,34612.378884,36083.912141,37862.775470,36904.993774,36238.757812,36243.781250,36224.816406,36164.046875,36217
6,000006,89602.604049,90865.507706,91455.010143,92263.179575,91192.984375,91298.570312,91249.820312,91231.750000,91243
7,000007,50504.610054,50595.346917,53087.618556,52502.538939,51620.992188,51656.078125,51627.031250,51581.906250,51621
8,000008,64983.259189,65745.354942,63962.524605,64361.126331,64809.320312,64850.421875,64796.332031,64831.410156,64821
9,000009,67768.413052,67144.742076,65642.277494,65179.928797,66500.023438,66538.476562,66474.851562,66548.679688,66515


## 後処理

### 1. 前回データ

unit_idで紐付け可能な前回データを持つものは、最新のデータで置き換えを行う。

In [24]:
df_test_prev =  df_test[['index', 'target_ym', 'building_id', 'unit_id', 'bukken_id']].merge(
    df_train[['unit_id', 'money_room', 'target_ym']], on='unit_id', how='left',
    suffixes=('', '_prev')
)

df_test_prev = df_test_prev[~df_test_prev['money_room'].isnull()]
df_test_prev = df_test_prev.sort_values(['index', 'target_ym_prev'], ascending=[True, False])
df_test_prev = df_test_prev.drop_duplicates(keep='first', subset=['index'])

dispmd(f'### DataFrame Shape = ROWS: {df_test_prev.shape[0]:,}, COLUMNS: {df_test_prev.shape[1]}')

disp(df_test_prev.head(20))

df_pred = df_pred.merge(df_test_prev, on='index', how='left')
df_pred['predict2'] = df_pred.apply(lambda row: row['predict'] if row['money_room'] is pd.NA else row['money_room'], axis=1)

disp(df_pred.head(20))


### DataFrame Shape = ROWS: 33,118, COLUMNS: 7

,index,target_ym,building_id,unit_id,bukken_id,money_room,target_ym_prev
3,000003,202207,a200797,6759472,1434580009728,51000,202001
15,000015,202201,a183716,36749916,1133100029831,55000,202107
49,000049,202307,a331341,28759216,37065700026072,39000,202001
61,000061,202201,a519619,4891002,31032240084654,36000,202107
71,000071,202207,a510625,20558985,1420310010545,47000,202001
75,000075,202201,a372685,18322497,37064260014088,47000,202107
81,000081,202301,a213601,18584144,1224080148010,47000,201907
84,000084,202201,a327302,28279172,31036290082652,36000,202007
106,000106,202201,a026630,17421601,37064310009283,40500,202007
113,000113,202207,a488521,31091338,32203060000053,35000,202107


,index,predict_lgb_2019,predict_lgb_1984,predict_cat_2019,predict_cat_1984,predict_ly2_0,predict_ly2_1,predict_ly2_2,predict_ly2_3,predict,target_ym,building_id,unit_id,bukken_id,money_room,target_ym_prev,predict2
0,000000,66353.469432,65277.613958,67082.149468,66273.108700,66283.406250,66337.367188,66289.851562,66300.359375,66302,<NA>,NaN,NaN,NaN,<NA>,<NA>,66302
1,000001,48904.993797,48891.658193,51485.666510,51400.380814,50108.445312,50143.226562,50116.316406,50060.031250,50107,<NA>,NaN,NaN,NaN,<NA>,<NA>,50107
2,000002,61123.626548,59031.901382,63186.928742,62939.201934,61563.875000,61622.210938,61586.625000,61546.765625,61579,<NA>,NaN,NaN,NaN,<NA>,<NA>,61579
3,000003,47214.090031,46360.806681,48290.079574,48048.734072,47416.265625,47439.554688,47404.750000,47381.824219,47410,202207,a200797,6759472.0,1.434580e+12,51000,202001,51000
4,000004,80877.416967,79140.103846,81861.133504,79829.764021,80536.476562,80614.593750,80558.406250,80595.109375,80576,<NA>,NaN,NaN,NaN,<NA>,<NA>,80576
5,000005,34612.378884,36083.912141,37862.775470,36904.993774,36238.757812,36243.781250,36224.816406,36164.046875,36217,<NA>,NaN,NaN,NaN,<NA>,<NA>,36217
6,000006,89602.604049,90865.507706,91455.010143,92263.179575,91192.984375,91298.570312,91249.820312,91231.750000,91243,<NA>,NaN,NaN,NaN,<NA>,<NA>,91243
7,000007,50504.610054,50595.346917,53087.618556,52502.538939,51620.992188,51656.078125,51627.031250,51581.906250,51621,<NA>,NaN,NaN,NaN,<NA>,<NA>,51621
8,000008,64983.259189,65745.354942,63962.524605,64361.126331,64809.320312,64850.421875,64796.332031,64831.410156,64821,<NA>,NaN,NaN,NaN,<NA>,<NA>,64821
9,000009,67768.413052,67144.742076,65642.277494,65179.928797,66500.023438,66538.476562,66474.851562,66548.679688,66515,<NA>,NaN,NaN,NaN,<NA>,<NA>,66515


### 2. 高額物件等の補正 

In [ ]:
df_pred['predict2'] = df_pred['predict2'].astype(int)
df_pred = df_pred.merge(df_test, on='index', how='left')


df_pred.loc[df_pred['building_id'].isin(['a011317', 'a015506', 'a177203', 'a419598' # 六本木ヒルズ
                                                   'a050077', #アークタワーズイースト
                                                   ]), 'predict2'] *= 1.055

df_pred.loc[df_pred['building_id'].isin(['a065919' # 六本木ファーストプラザ
                                                   ]), 'predict2'] *= 1.10

df_pred.loc[df_pred['building_id'].isin(['a033735' # ザ・レジデンス六本木
                                                   ]), 'predict2'] *= 1.07

df_pred.loc[(df_pred['predict2'] >= 500_000) & (df_pred['predict2'] < 1_000_000) & (~df_pred['building_id'].isin(['a011317', 'a015506', 'a177203', 'a419598', 'a050077', 
                                                   'a050077', #アークタワーズイースト
                                                   'a065919', # 六本木ファーストプラザ
                                                   'a033735' # ザ・レジデンス六本木
                                                   ]) & (df_pred['house_area'] <= 200)), 'predict2'] *= 1.045

# 追加Ver4
df_pred.loc[(df_pred['predict2'] >= 1_000_000) & (~df_pred['building_id'].isin(['a011317', 'a015506', 'a177203', 'a419598', 'a050077', 
                                                   'a050077', #アークタワーズイースト
                                                   'a065919', # 六本木ファーストプラザ
                                                   'a033735' # ザ・レジデンス六本木
                                                   ]) & (df_pred['house_area'] <= 200)), 'predict2'] *= 1.060

# パークタワー青山
df_pred.loc[df_pred['index'] == '079292', 'predict2'] *= 1.02


# 大分県国東市武蔵町内田 〇〇武蔵の4棟
df_pred.loc[df_pred['index'].isin(['184686', '310258']), 'predict2'] = 6_000

# 若竹荘
df_pred.loc[df_pred['index'].isin(['139510', '332787']), 'predict2'] = 15_000

# R28番館
df_pred.loc[df_pred['index'].isin(['102881']), 'predict2'] = 8_000


df_pred['predict2'] = df_pred['predict2'].astype(int)


now = dt.datetime.now().strftime('%Y%m%dT%H%M%S')
dispmd(f'### {now}')
disp(df_pred[['index', 'predict2']])

df_pred[['index', 'predict2']].to_csv(f'submission_{now}.csv', header=None, index=None)